# Setup

In [1]:
!pip install datasets transformers torch accelerate sacremoses sacrebleu --quiet
!pip install dataset --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.53 which is incompatible.


In [2]:
import torch, os
from transformers import MarianTokenizer, MarianMTModel
import pandas as pd
import sacrebleu
from tqdm import tqdm
from datasets import load_dataset


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["WANDB_DISABLED"] = "true"

# Load the Model and Tokenizer

In [4]:
model_name = '/kaggle/input/finetuned-opusmt-en-to-te-model'
tokenizer = MarianTokenizer.from_pretrained(model_name)
# Load the model
model = MarianMTModel.from_pretrained(model_name)
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(64110, 512, padding_idx=64109)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(64110, 512, padding_idx=64109)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

# Tatoeba Benchmark Evaluation

## Load Tatoeba dataset

In [5]:
# Load the dataset 
df = pd.read_csv('/kaggle/input/tatoeba-telugu/Tatoeba-telugu.csv')
english_sentences = df['English'].tolist()
telugu_sentences = df['Telugu'].to_list()

In [6]:
print(english_sentences[:10])
print(telugu_sentences[:10])

['A cat came out from under the desk.', 'Ah! How beautiful the Taj Mahal is!', 'Ah! How serene is this temple!', 'Are you feeling OK?', 'Are you for real?!', 'Are you guys doing well?', 'Are you guys OK?', 'Are you mad?', 'Are you prepared to do this?', 'Are you still afraid something might happen?']
['ఒక పిల్లి డెస్క్ కింద నుండి బయటకు వచ్చింది.', 'ఆహా! తాజ్ మహల్ ఎంత బాగుంది!', 'ఆహా! ఈ గుడి ఎంత ప్రశాంతంగా ఉంది!', 'వొంట్లో ఎలా వుంది', 'ఎమిటీ! ఇది నిజమా?', 'మీరు బాగున్నారా ?', 'మీరు బాగున్నారా ?', 'కోపమొచ్చిందా ?', 'ఇది చెయ్యడానికి సిద్దంగా వున్నావా', 'ఏమైనా అవుతుందని ఇంకా భయపడుతున్నావా ?']


## Calculate BLEU: Tatoeba

In [7]:
# Function to generate translation for a given input text
def generate_translation(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        output_ids = model.generate(input_ids)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

translations = []
references = []
for i in tqdm(range(0, len(english_sentences)), desc="Translating"):
    translations.append(generate_translation(english_sentences[i]))
    references.append([telugu_sentences[i]])

# Calculate BLEU score
bleu = sacrebleu.corpus_bleu(translations, references)
print(f"BLEU score: {bleu.score}")


Translating: 100%|██████████| 262/262 [02:29<00:00,  1.75it/s]

BLEU score: 24.446151121745064


# IN-22 Benchmark Evaluation

## Load IN-22 dataset

In [8]:
# download and load specific pairs
dataset = load_dataset("ai4bharat/IN22-Gen", "eng_Latn-tel_Telu", trust_remote_code=True)


Generating gen split: 0 examples [00:00, ? examples/s]

In [9]:
dataset

DatasetDict({
    gen: Dataset({
        features: ['id', 'context', 'source', 'url', 'domain', 'num_words', 'bucket', 'sentence_eng_Latn', 'sentence_tel_Telu'],
        num_rows: 1024
    })
})

In [10]:
english_sentences = dataset['gen']['sentence_eng_Latn']
telugu_sentences = dataset['gen']['sentence_tel_Telu']

# Convert them to lists
english_sentences = list(english_sentences)
telugu_sentences = list(telugu_sentences)

# Verify the first few elements of each list
print(english_sentences[:5])
print(telugu_sentences[:5])


['An appearance is a bunch of attributes related to the service person, like their shoes, clothes, tie, jewellery, hairstyle, make-up, watch, cosmetics, perfume, etc.', 'Ajanta, located in the Aurangabad District of Maharashtra has twenty-nine caitya and vihara caves decorated with sculptures and paintings from the first century B.C.E. to the fifth century C.E.', 'Body colour gets merged with the outer line, creating the effect of volume.', 'Ashoka started making extensive use of stone for sculptures and great monuments, whereas the previous tradition consisted of working with wood and clay.', 'Potatoes mixed in masalas, coated in besan batter and deep fried to perfection form this delicious and famous dish of Maharashtra.']
['బాహ్య రూపం అనేది సేవ చేసే వ్యక్తి తాలూకు బూట్లు, బట్టలు, టై, ఆభరణాలు, కేశాలంకరణ, అలంకరణ, గడియారం, సౌందర్య సాధనాలు,అత్తరు మొదలైనటువంటి వాటికి సంబంధించిన లక్షణాల సమాహారం.', 'మహారాష్ట్ర లోని ఔరంగాబాద్ జిల్లాలో నెలకొని ఉన్నఅజంతా, ఇరవై తొమ్మిది చైత్యా మరియు విహారా గుహ

In [11]:
from transformers import MarianTokenizer, MarianMTModel

model_name = '/kaggle/input/finetuned-opusmt-en-to-te-model'
tokenizer = MarianTokenizer.from_pretrained(model_name)
# Load the model
model = MarianMTModel.from_pretrained(model_name)
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(64110, 512, padding_idx=64109)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(64110, 512, padding_idx=64109)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

## Calculate BLEU: IN-22

In [12]:
# Function to generate translation for a given input text
def generate_translation(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        output_ids = model.generate(input_ids)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

translations = []
references = []
for i in tqdm(range(0, len(english_sentences)), desc="Translating"):
    translations.append(generate_translation(english_sentences[i]))
    references.append([telugu_sentences[i]])

# Calculate BLEU score
bleu = sacrebleu.corpus_bleu(translations, references)
print(f"BLEU score: {bleu.score}")


Translating: 100%|██████████| 1024/1024 [43:20<00:00,  2.54s/it]


BLEU score: 10.640850690356462


## Calculate chrF

In [13]:
translations = []
references = []
for i in tqdm(range(0, len(english_sentences)), desc="Translating"):
    translations.append(generate_translation(english_sentences[i]))
    references.append(telugu_sentences[i])  
# Calculate chrF score
chrf = sacrebleu.corpus_chrf(translations, references)
print(f"chrF score: {chrf.score}")

Translating: 100%|██████████| 1024/1024 [42:12<00:00,  2.47s/it]


chrF score: 5.3475935828877015
